In [1]:
import json
import csv
from functools import reduce

In [2]:
def carrega_dados(path:str = 'salarios.json') -> list[dict]:
    try:
        with open(path, 'r') as arquivo: # abre o arquivo especificado pelo caminho path no modo de leitura ('r')
            dados = arquivo.read()
            return json.loads(dados)
    except FileNotFoundError: # ocorre quando o arquivo especificado em open() não é encontrado
        return []

def salvar_dados(dados:list[dict], path:str = 'salarios.json') -> bool:
    try:
        with open(path, 'w') as arquivo:
            arquivo.write(json.dumps(dados))
            return True
    except Exception:
        return False

In [3]:
dados = carrega_dados()

In [4]:
def remover_info(dados, *chaves) -> list[dict]:
    return [{k: v for k, v in d.items() if k not in chaves} for d in dados]

chaves_a_remover = ['ano_de_trabalho', 'nivel_de_experiencia', 'cargo','salario_em_usd', 'residencia_do_funcionario', 'localizacao_da_empresa', 'tamanho_da_empresa' ]  # Pode ser uma lista com várias chaves
incremental = remover_info(dados, *chaves_a_remover)

print(incremental[0])
print(len(incremental))

{'id': 1}
219


In [35]:
def inserir_dado(dados: list[dict], incremental: list[dict]) -> bool:
    try:
        novo_registro = {
            'ano_de_trabalho': input('\nAno da informação: \n'),
            'nivel_de_experiencia': input('Qual a experiência: \n'),
            'cargo': input('Qual a profissão: \n'),
            'salario_em_usd': input('Qual o salário em dolar: \n'),
            'residencia_do_funcionario': input('Qual a residência : \n'),
            'localizacao_da_empresa': input('Qual a localização da empresa: \n'),
            'tamanho_da_empresa': input('Quantos funcionários: \n'),
            'id': len(incremental) + 1
        }

        dados.append(novo_registro)
        incremental.append({'id': len(incremental) + 1})

        print("Registro inserido com sucesso.")
        return True
    except Exception as e:
        print(f"Ocorreu um erro ao inserir o registro: {e}")
        return False

In [44]:
def atualizar_dado(dados):
    while True:
        try:
            id_atualizado = int(input("Insira o ID que se deseja atualizar (ou digite 9 para voltar ao menu): "))
        except ValueError:
            print("Por favor, insira um ID válido (número inteiro).")
            continue

        if id_atualizado == 9:
            print("Voltando para o menu principal.")
            return False

        encontrado = False
        for dado in dados:
            if dado.get("id") == id_atualizado:
                print("Registro atual:")
                print(dado)

                try:
                    new_ano_de_trabalho = input("Entre com novo ano: ")
                    new_nivel_de_experiencia = input("Entre com novo nivel de experiencia: ")
                    new_cargo = input("Entre com novo cargo: ")
                    new_salario_em_usd = input("Entre com novo salário em dolar: ")
                    new_residencia_do_funcionario = input("Entre com a residência do funcionário: ")
                    new_localizacao_da_empresa = input("Entre com localização da empresa: ")
                    new_tamanho_da_empresa = input("Entre com tamanho da empresa: ")
                except Exception as e:
                    print(f"Erro ao obter novos dados: {e}")
                    return False

                # Atualizando o dict
                dado["ano_de_trabalho"] = new_ano_de_trabalho
                dado["nivel_de_experiencia"] = new_nivel_de_experiencia
                dado["cargo"] = new_cargo
                dado["salario_em_usd"] = new_salario_em_usd
                dado["residencia_do_funcionario"] = new_residencia_do_funcionario
                dado["localizacao_da_empresa"] = new_localizacao_da_empresa
                dado["tamanho_da_empresa"] = new_tamanho_da_empresa

                print("Dicionário atualizado com sucesso.")
                encontrado = True
                break

        if not encontrado:
            print(f"Nenhum registro encontrado com o ID {id_atualizado}. Tente novamente.")

In [30]:
def apagar_dado(dados):
    while True:
        try:
            id_para_apagar = int(input("Qual ID deseja apagar (ou digite 9 para voltar ao menu): "))
        except ValueError:
            print("Por favor, insira um ID válido (número inteiro).")
            continue

        if id_para_apagar == 9:
            print("Voltando para o menu principal.")
            return False

        apagado = [dado for dado in dados if dado['id'] == id_para_apagar]

        if len(apagado) == 0:
            print('Não foi encontrado nenhum registro com o ID informado. Tente novamente.')
        else:
            print(f'Excluídos {len(apagado)} registros!')

            # Modificando a lista original
            dados[:] = [dic for dic in dados if dic['id'] != id_para_apagar]

            return True

In [8]:
def valida_opcoes(valor: str, opcoes: list) -> bool:
    # Verifica se o valor está presente na lista de opções.
    return valor in opcoes

In [22]:
def obter_opcoes(opcoes, msg='Opções'):
    # Construindo a mensagem com as opções disponíveis
    msg = f"\n{msg} ({' | '.join([f'{key} - {value}' for key, value in opcoes.items()])}):"

    while True:
        # Solicitando que o usuário insira uma opção
        valor = input(msg).upper()

        # Verificando se a opção inserida é válida usando a função valida_opcoes
        if valida_opcoes(valor, opcoes.keys()):
            break

        # Se a opção não for válida, gerando uma mensagem de erro
        msg = f'Entrada Inválida! As opções válidas são {", ".join(opcoes.keys())}\n' + msg

    # Retornando a opção válida inserida pelo usuário
    return valor

In [20]:
def obter_ano_de_trabalho(dados, msg='Anos') -> str:
    anos = sorted(list(set(dado['ano_de_trabalho'] for dado in dados)))
    while True:
        try:
            ano = input(f"{msg} ({' | '.join(anos)}): ")
            if ano in anos:
                break
            else:
                raise ValueError('Entrada Inválida! Dado não está na lista.')
        except ValueError as e:
            print(f'Erro: {e}\n')
    return ano

def obter_nivel_de_experiencia(dados, msg='Experiencias') -> str:
    experiencias = sorted(list(set(dado['nivel_de_experiencia'] for dado in dados)))
    while True:
        try:
            experiencia = input(f"{msg} ({' | '.join(experiencias)}): ").title()
            if experiencia in experiencias:
                break
            else:
                raise ValueError('Entrada Inválida! Dado não está na lista.')
        except ValueError as e:
            print(f'Erro: {e}\n')
    return experiencia

def obter_cargo(dados, msg='Cargo') -> str:
    cargos = sorted(list(set(dado['cargo'] for dado in dados)))
    while True:
        try:
            cargo = input(f"{msg} ({' | '.join(cargos)}): ")
            if cargo in cargos:
                break
            else:
                raise ValueError('Entrada Inválida! Dado não está na lista.')
        except ValueError as e:
            print(f'Erro: {e}\n')
    return cargo

def obter_residencia_do_funcionario(dados, msg='Residencia do Funcionario') -> str:
    funcionarios = sorted(list(set(dado['residencia_do_funcionario'] for dado in dados)))
    while True:
        try:
            funcionario = input(f"{msg} ({' | '.join(funcionarios)}): ").upper()
            if funcionario in funcionarios:
                break
            else:
                raise ValueError('Entrada Inválida! Dado não está na lista.')
        except ValueError as e:
            print(f'Erro: {e}\n')
    return funcionario

def obter_localizacao_da_empresa(dados, msg='Localizacao da Empresa') -> str:
    empresas = sorted(list(set(dado['localizacao_da_empresa'] for dado in dados)))
    while True:
        try:
            empresa = input(f"{msg} ({' | '.join(empresas)}): ").upper()
            if empresa in empresas:
                break
            else:
                raise ValueError('Entrada Inválida! Dado não está na lista.')
        except ValueError as e:
            print(f'Erro: {e}\n')
    return empresa

def obter_tamanho_da_empresa(dados, msg='tamanho_da_empresa') -> str:
    tamanhos = sorted(list(set(dado['tamanho_da_empresa'] for dado in dados)))
    while True:
        try:
            tamanho = input(f"{msg} ({' | '.join(tamanhos)}): ").upper()
            if tamanho in tamanhos:
                break
            else:
                raise ValueError('Entrada Inválida! Dado não está na lista.')
        except ValueError as e:
            print(f'Erro: {e}\n')
    return tamanho

In [11]:
# filtrando por ano -> a saída de cada função será a entrada da próxima
def filtro_ano_de_trabalho(item, opc) -> list[dict]:
    try:
        return [dado for dado in item if dado['ano_de_trabalho'] == opc]
    except TypeError:
        raise ValueError('Filtro de ano inválido.')
    except Exception as e:
        raise ValueError(f'Erro no filtro de ano: {e}')
# |
# v

# filtra por nível de experiencia:
def filtro_nivel_de_experiencia(item, opc) -> list[dict]:
    try:
        return [dado for dado in item if dado['nivel_de_experiencia'] == opc]
    except TypeError:
        raise ValueError('Filtro de ano inválido.')
    except Exception as e:
        raise ValueError(f'Erro no filtro de ano: {e}')

# |
# v

# filtra por tipo de cargo
def filtro_cargo(item, opc) -> list[dict]:
    try:
        return [dado for dado in item if dado['cargo'] == opc]
    except TypeError:
        raise ValueError('Filtro de ano inválido.')
    except Exception as e:
        raise ValueError(f'Erro no filtro de ano: {e}')

# |
# v

# filtra por localização do funcionário
def filtro_residencia_do_funcionario(item, opc) -> list[dict]:
    try:
        return [dado for dado in item if dado['residencia_do_funcionario'] == opc]
    except TypeError:
        raise ValueError('Filtro de localização inválido .')
    except Exception as e:
        raise ValueError(f'Erro no filtro de localização funcionário: {e}')
# |
# v

# filtra por localização da empresa
def filtro_localizacao_da_empresa(item, opc) -> list[dict]:
    try:
        return [dado for dado in item if dado['localizacao_da_empresa'] == opc]
    except TypeError:
        raise ValueError('Filtro de localização Empresa inválido .')
    except Exception as e:
        raise ValueError(f'Erro no filtro localização Empresa: {e}')

# |
# v

# filtra por tamanho da empresa
def filtro_tamanho_da_empresa(item, opc) -> list[dict]:
    try:
        return [dado for dado in item if dado['tamanho_da_empresa'] == opc]
    except TypeError:
        raise ValueError('Filtro de ano inválido.')
    except Exception as e:
        raise ValueError(f'Erro no filtro de ano: {e}')

In [32]:
def visualizar_dados(dados):
    if not dados:
        print("Não há dados para visualizar.")
        return

    for registro in dados:
        print("ID:", registro["id"])
        print("Ano de Trabalho:", registro["ano_de_trabalho"])
        print("Nível de Experiência:", registro["nivel_de_experiencia"])
        print("Cargo:", registro["cargo"])
        print("Salário em USD:", registro["salario_em_usd"])
        print("Residência do Funcionário:", registro["residencia_do_funcionario"])
        print("Localização da Empresa:", registro["localizacao_da_empresa"])
        print("Tamanho da Empresa:", registro["tamanho_da_empresa"])
        print("-" * 30)

In [12]:
def salvar_dados_csv(dados:list[dict], path:str = 'salirios_novos.json') -> bool:
    try:
        with open(path, 'w') as arquivo:
            arquivo.write(json.dumps(dados))
            return True
    except Exception:
        return False

In [13]:
def salvar_csv(filtros, estatisticas):
    header = [['Filtros Aplicados', 'Máximo', 'Mínimo', 'Média']]
    data =   [[', '.join(filtros), estatisticas[0], estatisticas[1], estatisticas[2]]]
    with open('estatisticas.csv', 'w', newline='', encoding='utf-8') as arquivo:
        escritor = csv.writer(arquivo, delimiter=';', lineterminator='\n')
        escritor.writerows(header+data)

In [43]:
def menu():
    opcao_entrada = {'1': 'Inserir', '2': 'Remover', '3': 'Atualizar', '4': 'Estatística', '9': 'Sair'}
    item = int(obter_opcoes(opcao_entrada, 'Escolha a opcao'))

    if item == 1:
        inserir_dado(dados, incremental)
        salvar_dados(dados, 'salarios.json')

    elif item == 2:
        visualizar_dados(dados)
        apagar_dado(dados)
        salvar_dados(dados, 'salarios.json')

    elif item == 3:
        visualizar_dados(dados)
        atualizar_dado(dados)
        salvar_dados(dados, 'salarios.json')

    elif item == 4:

        ano_de_trabalho = obter_ano_de_trabalho(dados)
        dados_finais = filtro_ano_de_trabalho(dados, ano_de_trabalho)
        contagem = len(dados_finais)
        print(f'Número de registros do ano {ano_de_trabalho}: {contagem}')

        nivel_de_experiencia = obter_nivel_de_experiencia(dados_finais)
        dados_finais = filtro_nivel_de_experiencia(dados_finais, nivel_de_experiencia)
        contagem = len(dados_finais)
        print(f'Número de registros desse nível de experiência: {contagem}')

        cargo = obter_cargo(dados_finais)
        dados_finais = filtro_cargo(dados_finais, cargo)
        contagem = len(dados_finais)
        print(f'Número de registros desse cargo: {contagem}')

        residencia_do_funcionario =obter_residencia_do_funcionario(dados_finais)
        dados_finais = filtro_residencia_do_funcionario(dados_finais, residencia_do_funcionario)
        contagem = len(dados_finais)
        print(f'Número de registros de funcionários que moram nesse local: {contagem}')

        localizacao_da_empresa =obter_localizacao_da_empresa(dados_finais)
        dados_finais = filtro_localizacao_da_empresa(dados_finais, localizacao_da_empresa)
        contagem = len(dados_finais)
        print(f'Número de empresas sediadas nesse local: {contagem}')

        tamanho_da_empresa = obter_tamanho_da_empresa(dados_finais)
        dados_finais = filtro_tamanho_da_empresa(dados_finais, tamanho_da_empresa)
        contagem = len(dados_finais)
        print(f'Número de registros de empresas desse porte: {contagem}')

        salarios = [float(x['salario_em_usd']) for x in dados_finais]
        maximo = max(salarios)
        minimo = min(salarios)
        media = sum(salarios)/len(salarios)

        filtros = [ano_de_trabalho, nivel_de_experiencia, cargo, residencia_do_funcionario, localizacao_da_empresa, tamanho_da_empresa]
        estatisticas = [maximo, minimo, media]

        salvar_csv(filtros, estatisticas)
        print(f'A média salarial é {media:.2f}')

    elif item == 9:
        return False
    return True

In [ ]:
while True:
    item = menu()
    salvar_dados(dados) #Salvandos os dados modificados
    if item == False:
        break
    if obter_opcoes({'S': 'Sim', 'N': 'Não'}, 'Deseja Sair') == 'S':
        break

In [16]:
def printar_csv(nome_arquivo):
    try:
        with open(nome_arquivo, 'r', newline='', encoding='utf-8') as arquivo:
            leitor = csv.reader(arquivo, delimiter=';')
            for linha in leitor:
                print(linha)
    except FileNotFoundError:
        print(f'O arquivo {nome_arquivo} não foi encontrado.')
    except Exception as e:
        print(f'Erro ao abrir o arquivo: {e}')

# Chamada da função para imprimir o conteúdo do arquivo CSV
printar_csv('estatisticas.csv')

['Filtros Aplicados', 'Máximo', 'Mínimo', 'Média']
['2023, Junior, Data Science Engineer, CH, CH, S', '104024.0', '104024.0', '104024.0']
